In [1]:
from constants import CHROMA_SETTINGS
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All, LlamaCpp
import os
import pandas as pd
import numpy as np
import math

In [2]:
embeddings_model_name = os.environ.get("EMBEDDINGS_MODEL_NAME")
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

In [3]:
def emb_func(text):
    return embeddings.embed_documents([text])[0]
def compute_cosine_similarity(embedding_1, embedding_2):
    embedding_1 = np.array(embedding_1)
    embedding_2 = np.array(embedding_2)
    sumxx = np.dot(embedding_1, embedding_1)
    sumyy = np.dot(embedding_2, embedding_2)
    sumxy = np.dot(embedding_1, embedding_2)
    return sumxy / math.sqrt(sumxx * sumyy)

In [22]:
df_acidente = pd.read_csv('input/Accident_N0_tratada_8070.csv')
df_relatos = pd.read_csv('input/Relatos_N0_tratada_SemDeverdRecusa.csv')

In [23]:
# Relatos - 2º Semestre 2021
df_relatos['embedding'] = df_relatos['Descricao_tratada'].apply(emb_func)


Teste Individual

In [29]:
# Acidentes - 2022
acidente = df_acidente['Descricao_tratada'].iloc[0]
print("Acidente: ", acidente)

variavel = embeddings.embed_documents([acidente])[0]
df_relatos['similarity'] = df_relatos['embedding'].apply(lambda emb: compute_cosine_similarity(variavel, emb))

#df_relatos.drop(columns='level_0', inplace=True)
#df_relatos.drop(columns='index', inplace=True)

df_similarity = df_relatos.sort_values("similarity", ascending=False).reset_index()
acima_50 = df_similarity['similarity'] > 0.5
resultados = df_similarity.loc[acima_50].head(10)
mean_similarity = round(resultados['similarity'].mean(),2) * 100

df = pd.DataFrame()
df['Descricao_tratada'] = resultados['Descricao_tratada']
df['similarity'] = resultados['similarity']

print(mean_similarity)

pd.set_option('display.max_colwidth', None)
df

Acidente:  durante retirada isolamento autoclave arcc chapas galvanizadas contato involuntario panturrilha direita
80.0


,Descricao_tratada,similarity
0,isolamento termico autoclave arca piso terreo encontra pendurado apresentando risco queda,0.830070
1,entrada autoclave arca sem placa sinalizacao espaco confinado,0.829561
2,chuveiro emergencia lampada queimada ultimo piso area autoclaves parte cima arcc,0.820140
3,dar inicio montagem equipamentos hidrojateamento autoclaves observado tramas malha aco desfiadas,0.807726
4,chapas protecao termica solta plataforma proximo escada autoclaves apresentando risco queda nivel diferente,0.797910
5,acesso autoclaves sem iluminacao adequada circulacao pessoas operadores terceirosas,0.796973
6,iluminacao chuveiro emergencia queimada lado escada acesso autoclaves,0.792427
7,vazamento autoclave sem isolamento identificacao,0.784374
8,terceiro subindo escada acesso autoclaves viseira levantada sem uso corrimao,0.784060
9,chapa protecao movel autoclave arca sem suporte grapa fixa mesma risco queda chapa piso inferior queda pessoas,0.782997


Teste Base de Acidentes

In [14]:
for i, acidente in enumerate(df_acidente['Descricao_tratada']):
    variavel = embeddings.embed_documents([acidente])[0]
    df_relatos['similarity'] = df_relatos['embedding'].apply(lambda emb: compute_cosine_similarity(variavel, emb))
    df_similarity = df_relatos.sort_values("similarity", ascending=False).reset_index()
    df_similarity = df_similarity.drop('index', axis=1)
    acima_50 = df_similarity['similarity'] > 0.5          # Seleciona todos os relatos com 50% de similariedade
    resultados = df_similarity.loc[acima_50].head(10)     # Seleciona os 10 primeiros relatos
    mean_similarity = round(resultados['similarity'].mean(),2) * 100   # Faz do vaqlor 
    df_acidente.at[i, 'Mean_Similarity'] = round(resultados['similarity'].mean(),2) * 100

pd.set_option('display.max_colwidth', None)
df_acidente.sort_values("Mean_Similarity", ascending=False).reset_index()
df_acidente

,Descricao_tratada,Mean_Similarity
0,realizando remocao conjunto acionamento decantador auxilio talha manual movimentacao engrenagem dentada eixo prensando dedo,60.0
1,realizar acompanhamento tecnico instalacao bomba hbcc desequilibrou batendo punho esquerdo grelha protecao canaleta,63.0
2,realizar atividade extracao tubo trocador calor jxhe levantou espiga extratora posicionar trava nesse momento espiga deslocou ocasionando prensamento,64.0
3,movimentar lanca gas natural calcinador tras desprendimento mesma seguido projecao oxido quente atingindo emprego,62.0
4,durante abertura tampa inferior trocador efshc vazamento licor caustico temperatura ambiente atingindo perna,63.0
5,descer andaime desequilibrouse desnivel existente piso gerando torcao pe esquerdo,79.0
6,durante atividade rota inspecao sala eletrica hseb deslocar corredor veio torcer pe,69.0
7,realizar carregamento tubos andaime carroceria veiculo hr prensou dedo guarda veiculo tubo,70.0
8,durante deslocamento area proximo trocador hxhe sentiu desconforto rosto,67.0
9,durante retirada isolamento autoclave arcc chapas galvanizadas contato involuntario panturrilha direita,80.0
